# 赛题理解  
### 赛题名称：零基础入门NLP之新闻文本分类  
### 赛题目标：通过这道赛题可以引导大家走入自然语言处理的世界，带大家接触NLP的预处理、模型构建和模型训练等知识点。  
### 赛题任务：赛题以自然语言处理为背景，要求选手对新闻文本进行分类，这是一个典型的字符识别问题。  

# 学习目标  
理解赛题背景与赛题数据  
完成赛题报名和数据下载，理解赛题的解题思路  

# 赛题数据  
赛题以匿名处理后的新闻数据为赛题数据，数据集报名后可见并可下载。赛题数据为新闻文本，并按照字符级别进行匿名处理。整合划分出14个候选分类类别：财经、彩票、房产、股票、家居、教育、科技、社会、时尚、时政、体育、星座、游戏、娱乐的文本数据。  
赛题数据由以下几个部分构成：训练集20w条样本，测试集A包括5w条样本，测试集B包括5w条样本。为了预防选手人工标注测试集的情况，我们将比赛数据的文本按照字符级别进行了匿名处理。  

# 数据标签  
在数据集中标签的对应的关系如下：{'科技': 0, '股票': 1, '体育': 2, '娱乐': 3, '时政': 4, '社会': 5, '教育': 6, '财经': 7, '家居': 8, '游戏': 9, '房产': 10, '时尚': 11, '彩票': 12, '星座': 13}  

# 数据读取
使用Pandas库完成数据读取操作，并对赛题数据进行分析。 

In [4]:
!pip install --upgrade pip
!pip install pandas==1.0.5 -i https://mirrors.huaweicloud.com/repository/pypi/simple

    100% |████████████████████████████████| 1.5MB 74.9MB/s ta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 10.1 MB 12.9 MB/s eta 0:00:01B/s eta 0:00:01 12.9 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [7]:
import pandas as pd
df = pd.read_csv('./NLP_data_list_0715.csv',encoding="gbk")
df.head()

,file,description,size,link
0,train_set.csv.zip,训练集数据（选手需要自行解压）,236.11MB,https://tianchi-competition.oss-cn-hangzhou.al...
1,test_a.csv.zip,测试集A榜数据（选手需要自行解压）,59.12MB,https://tianchi-competition.oss-cn-hangzhou.al...
2,test_a_sample_submit.csv,测试集A榜提交样例,97.66KB,https://tianchi-competition.oss-cn-hangzhou.al...


In [15]:
!wget https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531810/train_set.csv.zip
!wget https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531810/test_a.csv.zip
!wget https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531810/test_a_sample_submit.csv
!unzip test_a.csv.zip
!unzip train_set.csv.zip
!rm -r test_a.csv.zip
!rm -r train_set.csv.zip

--2020-07-20 15:20:29--  https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531810/train_set.csv.zip
Resolving proxy-notebook.modelarts-dev-proxy.com (proxy-notebook.modelarts-dev-proxy.com)... 192.168.0.172
Connecting to proxy-notebook.modelarts-dev-proxy.com (proxy-notebook.modelarts-dev-proxy.com)|192.168.0.172|:8083... connected.
Proxy request sent, awaiting response... 200 OK
Length: 247578940 (236M) [application/zip]
Saving to: ‘train_set.csv.zip’

train_set.csv.zip   100%[===================>] 236.11M  51.3MB/s    in 4.7s    

2020-07-20 15:20:35 (49.9 MB/s) - ‘train_set.csv.zip’ saved [247578940/247578940]

--2020-07-20 15:20:35--  https://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531810/test_a.csv.zip
Resolving proxy-notebook.modelarts-dev-proxy.com (proxy-notebook.modelarts-dev-proxy.com)... 192.168.0.172
Connecting to proxy-notebook.modelarts-dev-proxy.com (proxy-notebook.modelarts-dev-proxy.com)|192.168.0.172|:8083... connected.
Proxy request sent, awaiting res

In [3]:
#读取测试集数据并查看
import pandas as pd
df_test = pd.read_csv('./test_a.csv')
df_test.head(10)

,text
0,5399 3117 1070 4321 4568 2621 5466 3772 4516 2...
1,2491 4109 1757 7539 648 3695 3038 4490 23 7019...
2,2673 5076 6835 2835 5948 5677 3247 4124 2465 5...
3,4562 4893 2210 4761 3659 1324 2595 5949 4583 2...
4,4269 7134 2614 1724 4464 1324 3370 3370 2106 2...
5,4061 3792 2434 6065 5612 2738 1008 2252 5620 9...
6,307 768 600 7539 920 1362 2109 6758 3223 5430 ...
7,3770 2461 2151 4525 5708 4436 7349 4436 5422 2...
8,6235 6248 5620 1919 4097 7539 6122 1903 4893 6...
9,1141 4411 6902 1995 6054 2035 2112 6139 2717 2...


In [5]:
#读取数据集数据并查看
df_train=pd.read_csv('./train_set.csv',sep='\t')
df_train.head(10)

,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...
5,9,3819 4525 1129 6725 6485 2109 3800 5264 1006 4...
6,3,307 4780 6811 1580 7539 5886 5486 3433 6644 58...
7,10,26 4270 1866 5977 3523 3764 4464 3659 4853 517...
8,12,2708 2218 5915 4559 886 1241 4819 314 4261 166...
9,3,3654 531 1348 29 4553 6722 1474 5099 7541 307 ...


In [6]:
#读取样例
df_sample=pd.read_csv('./test_a_sample_submit.csv')
df_sample.head(10)

,label
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [7]:
#通过sklearn完成f1_score计算：
from sklearn.metrics import f1_score
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]
f1_score(y_true, y_pred, average='macro')

0.26666666666666666

# 解题思路  
赛题思路分析：赛题本质是一个文本分类问题，需要根据每句的字符进行分类。但赛题给出的数据是匿名化的，不能直接使用中文分词等操作，这个是赛题的难点。  
  
因此本次赛题的难点是需要对匿名字符进行建模，进而完成文本分类的过程。由于文本数据是一种典型的非结构化数据，因此可能涉及到特征提取和分类模型两个部分。为了减低参赛难度，我们提供了一些解题思路供大家参考：  

### 思路1：TF-IDF + 机器学习分类器  
直接使用TF-IDF对文本提取特征，并使用分类器进行分类。在分类器的选择上，可以使用SVM、LR、或者XGBoost。  
  
### 思路2：FastText  
FastText是入门款的词向量，利用Facebook提供的FastText工具，可以快速构建出分类器。  
  
### 思路3：WordVec + 深度学习分类器  
WordVec是进阶款的词向量，并通过构建深度学习分类完成分类。深度学习分类的网络结构可以选择TextCNN、TextRNN或者BiLSTM。  
  
### 思路4：Bert词向量  
Bert是高配款的词向量，具有强大的建模学习能力。  